# Working With MosaicJSON

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/developmentseed/titiler/master?filepath=docs%2Fexamples%2FWorking_with_MosaicJSON.ipynb)

### MosaicJSON

MosaicJSON is a specification created by DevelopmentSeed which aims to be an open standard for representing metadata about a mosaic of Cloud-Optimized GeoTIFF (COG) files.


> MosaicJSON can be seen as a Virtual raster (see GDAL's VRT) enabling spatial and temporal processing for a list of Cloud-Optimized GeoTIFF.

Ref:https://github.com/developmentseed/mosaicjson-spec


### Data

For this demo, we are going to use CloudOptimized GeoTIFF from Digitalglobe opendata: https://www.digitalglobe.com/ecosystem/open-data


### Endpoint

By default, TiTiler has a complete `mosaicjson` endpoint. For this demo we are going to use a slightly modifed version hosted by developmentseed at `https://api.cogeo.xyz`

Docs: https://api.cogeo.xyz/docs#/MosaicJSON


## Requirements

To be able to run this notebook you'll need the following requirements:
- rasterio
- folium
- requests
- tqdm
- BeautifullSoup (webpage parsing) 
- rio-tiler (2.0b8) (Optional)
- cogeo-mosaic (Optional)

`pip install rasterio folium requests tqdm bs4 requests`
`pip install rio-tiler cogeo-mosaic --pre`

In [ ]:
# Uncomment this line if you need to install the dependencies
#!pip install rasterio folium requests tqdm bs4 requests rio-tiler`
#!pip install cogeo-mosaic --pre

## Get the Data

In [ ]:
import os
import json
import rasterio
import requests
import bs4 as BeautifulSoup

from concurrent import futures
from rio_tiler.io import COGReader
from rasterio.features import bounds as featureBounds

import bs4 as BeautifulSoup

from folium import Map, TileLayer, GeoJson

### 1. Fetch and parse page

In [ ]:
url = "https://www.digitalglobe.com/ecosystem/open-data/california-colorado-fires"

# Read Page
r = requests.get(url)
soup = BeautifulSoup.BeautifulSoup(r.text)
s = soup.findAll('textarea')[0]

### 2. Find GeoTIFF Urls

In [ ]:
list_file = [line.lstrip() for line in s.contents[0].splitlines() if line.endswith(".tif")]

files = [
    dict(
        date=l.split("/")[6],
        tags=[l.split("/")[5]],
        path=l,
        sceneid=l.split("/")[7],
        preview=f"https://api.discover.digitalglobe.com/show?id={l.split('/')[7]}&f=jpeg",
        event=l.split("/")[4],
    )
    for l in list_file
]

files = sorted(files, key=lambda x:x["date"])

print(f"Number of GeoTIFF: {len(list_file)}")

### 3. Pre/Post event

In [ ]:
pre_event = list(filter(lambda x: x["tags"] == ["pre-event"], files))
post_event = list(filter(lambda x: x["tags"] == ["post-event"], files))

print(f"Number of Pre Event COG: {len(pre_event)}")
print(f"Number of Post Event COG: {len(post_event)}")

### 4. Create Features and Viz (Optional)

Read each file geo metadata

In [ ]:
def worker(meta):
    try:
        with COGReader(meta["path"]) as cog:
            wgs_bounds = cog.bounds
    except:
        return {}

    return {
        "geometry": {
            "type": "Polygon",
            "coordinates": [
                [
                    [wgs_bounds[0], wgs_bounds[3]],
                    [wgs_bounds[0], wgs_bounds[1]],
                    [wgs_bounds[2], wgs_bounds[1]],
                    [wgs_bounds[2], wgs_bounds[3]],
                    [wgs_bounds[0], wgs_bounds[3]]
                ]
            ]
        },
        "properties": meta,
        "type": "Feature"
    }


    
with futures.ThreadPoolExecutor(max_workers=5) as executor:
    features = [r for r in executor.map(worker, post_event) if r]

In [ ]:
geojson = {'type': 'FeatureCollection', 'features': features}

bounds = featureBounds(geojson)

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=6
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)
m

### 5. Create Mosaic

In [ ]:
titiler_endpoint = "https://api.cogeo.xyz"  # Devseed temporary endpoint

# !!! Update this !!!  
username = "anonymous"

layername = "dgopendata_CAfire_2020_post"  # WARNING, you can overwrite Mosaics

###### 5.1. Create Token

Note: Right now everyone can create a token to upload or create a mosaic in DevSeed infrastructure

Docs: https://api.cogeo.xyz/docs#/Token/create_token_tokens_create_post

In [ ]:
r = requests.post(
    f"{titiler_endpoint}/tokens/create",
    json={
        "username": username,
        "scope": ["mosaic:read", "mosaic:create"]
    }
).json()
token = r["token"]
print(token)

###### 5.2. Create Mosaic

Docs: https://api.cogeo.xyz/docs#/MosaicJSON/create_mosaic_mosaicjson_create_post

In [ ]:
responses
from rio_tiler.io import COGReader
with COGReader(post_event[0]["path"]) as cog:
    info = cog.info()
    print(info.minzoom)
    print(info.maxzoom)

In [ ]:
from cogeo_mosaic.mosaic import MosaicJSON

# We are creating the mosaicJSON using the features we created earlier
# by default MosaicJSON.from_feature will look in feature.properties.path to get the path of the dataset
mosaicdata = MosaicJSON.from_features(features, minzoom=10, maxzoom=18)

r = requests.post(
    f"{titiler_endpoint}/mosaicjson/upload",
    json={
        "username": username,
        "layername": layername,
        "mosaic": mosaicdata.dict(exclude_none=True)
    },
    params={
        "access_token": token,
    }
).json()

print(r)

###### 5.3. Display Tiles

Docs: https://api.cogeo.xyz/docs#/MosaicJSON/tilejson_mosaicjson__layer__tilejson_json_get

In [ ]:
r = requests.get(
    f"{titiler_endpoint}/mosaicjson/{username}.{layername}/tilejson.json",
).json()
print(r)

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="DigitalGlobe OpenData"
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
tiles.add_to(m)
geo_json.add_to(m)
m